In [13]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
import re
from nltk import TweetTokenizer
from IPython.display import display

In [2]:
# Load data
data = {}
data['train'] = pd.read_csv('../../hw3/train_mod.csv')
data['train']

,Unnamed: 0,tweet,hate,offensive,neither,tweet_processed
0,0,"@HarveyLevinTMZ @sly309 they want what I got, ...",1,0,0,"<mention> <mention> they want what i got , i w..."
1,1,@HeauxmerSimpson I'm jus tryna vaca away from ...,1,0,0,<mention> i'm jus tryna vaca away from the nig...
2,2,@HeilSidious Sion is a faggot. Very meh ganks ...,1,0,0,<mention> sion is a faggot . very meh ganks pr...
3,3,@Herfarm @SteveWorks4You @TheDemocrats your lo...,1,0,0,<mention> <mention> <mention> your low info re...
4,4,@Herfarm Move out of our country teabaggers!,1,0,0,<mention> move out of our country teabaggers !
5,5,@HollowDaDonLOM those god damn chinks. http://...,1,0,0,<mention> those god damn chinks . http://t.co/...
6,6,@Hovaa_ #niggerfood #niggerperson #you #are #a...,1,0,0,<mention> #niggerfood #niggerperson #you #are ...
7,7,@Hovaa_ ok wat ever u say whitey. u prolly nev...,1,0,0,<mention> ok wat ever u say whitey . u prolly ...
8,8,@Hovaa_ shut up lizard faggot nigger cunt,1,0,0,<mention> shut up lizard faggot nigger cunt
9,9,@Hovaa_ ya I know all the slang I'm racist I h...,1,0,0,<mention> ya i know all the slang i'm racist i...


## Preprocess

In [3]:
def preprocess_tweet(text):
    
    # Remove @mentions
    p = re.sub(r'@\w+', '<mention>', text)
    
    # tokenize
    tokenizer = TweetTokenizer()
    p = ' '.join(tokenizer.tokenize(p.lower()))
    
    return p

In [4]:
data['train']['tweet_processed'] = data['train']['tweet'].map(lambda x: preprocess_tweet(x))

In [41]:
data['train']['tweet_processed'].head()

0    <mention> <mention> they want what i got , i want what they got , america.they got the word nigger , i want it to                           
1    <mention> i'm jus tryna vaca away from the niggers bro .                                                                                    
2    <mention> sion is a faggot . very meh ganks pre - 6 and post - 6 his ganks are still pretty meh he's just a tank , nothing special imo mao +
3    <mention> <mention> <mention> your low info redneck dumbasses vote for these corporate shills that idiots like you believe #dummies         
4    <mention> move out of our country teabaggers !                                                                                              
Name: tweet_processed, dtype: object

In [42]:
data['train'].to_csv('../../hw3/train_mod.csv')

# Logistic regression baseline

In [5]:
vec = CountVectorizer(min_df=2)
vec.fit(data['train'].tweet_processed.tolist())

bow = {}
bow['train'] = vec.transform(data['train'].tweet_processed.tolist())
bow['train'].shape

(21083, 8534)

In [14]:
# Hate speech classification
x = {}
y = {}
x['train'] = bow['train']
y['train'] = data['train'].hate

clf = LogisticRegression()
# scores = cross_val_score(clf, x['train'], y['train'], cv=5)
scores = cross_val_score(clf, x['train'], y['train'], cv=5, scoring=make_scorer(f1_score))
# print(f'Accuracy: {scores.mean(): .1%}')
print(f'F1: {scores.mean(): .1%}')

F1:  25.7%


In [24]:
# Offensive language classification
x = {}
y = {}
x['train'] = bow['train']
y['train'] = data['train'].offensive

clf = LogisticRegression()
scores = cross_val_score(clf, x['train'], y['train'], cv=5)
print(f'Accuracy: {scores.mean(): .1%}')

Accuracy:  89.2%


## Error analysis

In [29]:
# Train on full dataset (hate speech) and see errors
clf = LogisticRegression()
y['train'] = data['train'].hate
clf.fit(x['train'], y['train'])
preds = clf.predict(x['train'])
preds.shape

(21083,)

In [30]:
false_pos = []
false_neg = []

for i, (gold, pred) in enumerate(zip(data['train'].hate.tolist(), preds)):
    if pred != gold:
        if gold == 0:
            false_pos.append(i)
        elif gold == 1:
            false_neg.append(i)
        else:
            "What???"
            
print(len(false_pos))
print(len(false_neg))

63
695


In [33]:
pd.set_option('display.max_colwidth', -1)

In [44]:
print("False positives:")
display(data['train'].loc[false_pos, ['tweet_processed', 'hate', 'offensive', 'neither']])

False positives:


,tweet_processed,hate,offensive,neither
61,<mention> dont favorite me you faggot,0,1,0
63,<mention> shut the fuck up faggot ass knicks cock sucker your opinion is irrelevant fag,0,1,0
658,<mention> go to sleep you fag !,0,1,0
668,"<mention> <mention> one charlotte store , but of course they are faggots",0,1,0
753,<mention> just played against a heimer support . biggest faggot ever . stupid cheese champion .,0,1,0
772,<mention> shut up faggot,0,1,0
794,<mention> #fag fuckin pirelli tire black ass nigga whit a white boy shirt on but my nigga did loosen up a lil bit . #bowbow,0,1,0
846,<mention> i knew farquaad a faggot,0,1,0
1027,"<mention> <mention> <mention> in other words you are accustomed to niggers so muchyour almighty one can be next to u , but yo nigger",0,1,0
1471,<mention> <mention> hey faggot spell faggot right you dipshit,0,1,0


In [45]:
print("False negatives:")
display(data['train'].loc[false_pos, ['tweet_processed', 'hate', 'offensive', 'neither']])

False negatives:


,tweet_processed,hate,offensive,neither
61,<mention> dont favorite me you faggot,0,1,0
63,<mention> shut the fuck up faggot ass knicks cock sucker your opinion is irrelevant fag,0,1,0
658,<mention> go to sleep you fag !,0,1,0
668,"<mention> <mention> one charlotte store , but of course they are faggots",0,1,0
753,<mention> just played against a heimer support . biggest faggot ever . stupid cheese champion .,0,1,0
772,<mention> shut up faggot,0,1,0
794,<mention> #fag fuckin pirelli tire black ass nigga whit a white boy shirt on but my nigga did loosen up a lil bit . #bowbow,0,1,0
846,<mention> i knew farquaad a faggot,0,1,0
1027,"<mention> <mention> <mention> in other words you are accustomed to niggers so muchyour almighty one can be next to u , but yo nigger",0,1,0
1471,<mention> <mention> hey faggot spell faggot right you dipshit,0,1,0
